# Derivation of the variational symmetries for the Fisher KKP
*Date:* 2023-02-01,<br>
*Written by:* Johannes Borgqvist.<br>
Ok, we are studying the travelling wave version of the dimensionless Fisher-KKP model:
$$u_{zz}=-\left(cu_{z}+u(1-u)\right)=f(u,u_{z}).$$
We have derived the following family of *Lagrangians*:
\begin{equation}
    L{\left(z,u,u_{z}\right)}=\exp{(cz)}\left(\dfrac{u_{z}^{2}}{2}+(u(1-u)+\phi'(u))\dfrac{u_{z}}{c}+\phi(u)\right)\,,\quad\phi\in\mathcal{C}^{2}(\mathbb{R}).
    \label{eq:Lagrangian}
\end{equation}
Moreover, any infinitesimals $\xi{(z,u)}$ and $\eta{(z,u)}$ as well as the function $G{(u)}$ which satisfies the linearised symmetry condition:
$$\dfrac{\partial L}{\partial u}\eta+\dfrac{\partial L}{\partial u_{z}}\left(\dfrac{\partial\eta}{\partial z}+u_{z}\left(\dfrac{\partial\eta}{\partial u}-\dfrac{\partial\xi}{\partial z}\right)-u_{z}^{2}\dfrac{\partial\xi}{\partial u}\right)+L\left(\dfrac{\partial\xi}{\partial z}+c\xi\right)+u_{z}L\dfrac{\partial\xi}{\partial u}=\frac{\partial G}{\partial t}+u_{z}\frac{\partial G}{\partial u}\,,$$
generate symmetries and, better still, they have a constant of motion associated with them.

Now, Feng has shown that for the wave speed 
\begin{equation}
c=\dfrac{5}{\sqrt{6}}
\label{eq:feng_wave}
\end{equation}
meaning that our model is given by
$$u_{zz}=-\left(\dfrac{5}{\sqrt{6}}u_{z}+u(1-u)\right)=f(u,u_{z})$$
has a symmetry generated by
\begin{equation}
X=-\dfrac{\sqrt{3}}{\sqrt{2}}\exp{\left(\dfrac{z}{\sqrt{6}}\right)}\partial_{z}+\exp{\left(\dfrac{z}{\sqrt{6}}\right)}u\partial_{u}.
\label{eq:symmetry}
\end{equation}
Let's see if we can derive a constant of motion for these types of symmetries. 
### Load SymPy

In [1]:
from sympy import *
from sympy.solvers.pde import pdsolve

### Define Lagrangian and linearised symmetry condition

In [2]:
# Variables
z,u,u_z = symbols('z u u_z')
# Travelling wave constant
c = symbols('c')
c = 5/sqrt(6)
# Allocate our dear functions
G = Function('G')(z,u)
xi = Function('xi')(z,u)
eta = Function('eta')(z,u)
L = Function('L')(z,u,u_z)
# Define our arbitrary function
phi = Function('phi')(u)
# Ok, define the Lagrangian
L = ((u_z**2)/(2))+(u*(1-u)+Derivative(phi,u,1).doit())*(u_z/c)+phi
L = exp(c*z)*L
# Define our linearised symmetry condition term by term
lin_sym = Derivative(L,u,1).doit()*eta
factor = Derivative(eta,z,1).doit() + u_z*(Derivative(eta,u,1).doit()-Derivative(xi,z,1).doit()) - (u_z**2)*Derivative(xi,u,1).doit()
lin_sym+=factor*Derivative(L,u_z,1).doit()
lin_sym += Derivative(L,z,1).doit()*xi
lin_sym += L*(Derivative(xi,z,1).doit()+u_z*Derivative(xi,u,1).doit())
lin_sym += -((Derivative(G,z,1).doit())+(u_z*Derivative(G,u,1).doit()))
# Let's simplify this expression a wee bit
lin_sym = expand(simplify(lin_sym/exp(c*z)))
# Make a copy just to be safe
lin_sym_copy = lin_sym

### Validate Lagrangian
Again, if we assume that $u_{zz}=f(u,u_{z})$, then we need to make sure that the Lagrangian satisfies the following condition:

\begin{equation}
\dfrac{\partial^{2}L}{\partial z\partial u_{z}}+u_{z}\dfrac{\partial^{2}L}{\partial u\partial u_{z}}+f\dfrac{\partial^{2}L}{\partial u_{z}^{2}}=\dfrac{\partial L}{\partial u}.
\label{eq:Lagrangian_condition}
\end{equation}

In [3]:
# Define the Fisher KKP
f = -(c*u_z+u*(1-u))
# Define the RHS of the expression
RHS = Derivative(L,u,1).doit()
# Define the very common derivative
dLduz = Derivative(L,u_z,1).doit()
# Define the LHS
LHS = Derivative(dLduz,z,1).doit()+u_z*Derivative(dLduz,u,1).doit() + f*Derivative(dLduz,u_z,1).doit()
# Now, if the Lagrangian is correctly defined, then LHS-RHS=0
print("Controlling Lagrangian...")
if simplify(LHS-RHS)==0:
    print("Correctly defined Lagrangian!")
else:
    print("Sorry, incorrectly defined Lagrangian!")

Controlling Lagrangian...
Correctly defined Lagrangian!


### Setup our infinitesimals

In [4]:
# Setup infinitesimals using Feng's formulae
xi_Feng = -((sqrt(3))/(sqrt(2)))*exp(z/sqrt(6))
eta_Feng = exp(z/sqrt(6))*u
# Insert them into the linearised symmetry condition
lin_sym = expand(lin_sym.subs(xi,xi_Feng).subs(eta,eta_Feng).doit())

### Split up linearised symmetry condition with respect to monomials $\{1,u_{z},u_{z}^{2},u_{z}^{3}\}$

In [5]:
# Define the monomials
monomials = [u_z**3, u_z**2, u_z, 1 ]
# Allocate a list for the determining equations
det_eq = []
# Loop over and extract equations
for monomial in monomials:
    # Update the copy of the linearised symmetry condition
    lin_sym_copy = lin_sym
    # Allocate the equation at hand
    temp_eq = 0
    # Extract our lovely determining equation
    if monomial == 1:
        temp_eq = expand(lin_sym_copy.subs(u_z,0))
    else:
        temp_eq = expand(lin_sym_copy.coeff(monomial).subs(u_z,0))
    # Remove any annoying denominator which might be there
    temp_eq,denom = fraction(temp_eq)
    # Append the darn determining equation
    det_eq.append((monomial,temp_eq))
# Let's do some fancy printing
equation_string = "\\begin{align}\n"
for eq_temp in det_eq:
    equation_string += latex(eq_temp[0]) + ":&" + latex(eq_temp[1]) + "&=0,\\\\\n"
equation_string += "\\end{align}\n"
# Prompt to the user
print("The determining equations:\n\n")
print(equation_string)


The determining equations:


\begin{align}
u_{z}^{3}:&0&=0,\\
u_{z}^{2}:&0&=0,\\
u_{z}:&- \frac{\sqrt{6} u^{2} e^{\frac{\sqrt{6} z}{6}}}{10} + \frac{\sqrt{6} u e^{\frac{\sqrt{6} z}{6}} \frac{d^{2}}{d u^{2}} \phi{\left(u \right)}}{5} + \frac{\sqrt{6} u e^{\frac{\sqrt{6} z}{6}}}{15} - \frac{3 \sqrt{6} e^{\frac{\sqrt{6} z}{6}} \frac{d}{d u} \phi{\left(u \right)}}{10} - e^{- \frac{5 \sqrt{6} z}{6}} \frac{\partial}{\partial u} G{\left(z,u \right)}&=0,\\
1:&- \frac{u^{3} e^{\frac{\sqrt{6} z}{6}}}{5} + \frac{u^{2} e^{\frac{\sqrt{6} z}{6}}}{5} + \frac{6 u e^{\frac{\sqrt{6} z}{6}} \frac{d}{d u} \phi{\left(u \right)}}{5} - 3 \phi{\left(u \right)} e^{\frac{\sqrt{6} z}{6}} - e^{- \frac{5 \sqrt{6} z}{6}} \frac{\partial}{\partial z} G{\left(z,u \right)}&=0,\\
\end{align}



The determining equations:


\begin{align}
u_{z}^{3}:&0&=0,\\
u_{z}^{2}:&0&=0,\\
u_{z}:&- \frac{\sqrt{6} u^{2} e^{\frac{\sqrt{6} z}{6}}}{10} + \frac{\sqrt{6} u e^{\frac{\sqrt{6} z}{6}} \frac{d^{2}}{d u^{2}} \phi{\left(u \right)}}{5} + \frac{\sqrt{6} u e^{\frac{\sqrt{6} z}{6}}}{15} - \frac{3 \sqrt{6} e^{\frac{\sqrt{6} z}{6}} \frac{d}{d u} \phi{\left(u \right)}}{10} - e^{- \frac{5 \sqrt{6} z}{6}} \frac{\partial}{\partial u} G{\left(z,u \right)}&=0,\\
1:&- \frac{u^{3} e^{\frac{\sqrt{6} z}{6}}}{5} + \frac{u^{2} e^{\frac{\sqrt{6} z}{6}}}{5} + \frac{6 u e^{\frac{\sqrt{6} z}{6}} \frac{d}{d u} \phi{\left(u \right)}}{5} - 3 \phi{\left(u \right)} e^{\frac{\sqrt{6} z}{6}} - e^{- \frac{5 \sqrt{6} z}{6}} \frac{\partial}{\partial z} G{\left(z,u \right)}&=0.\\
\end{align}
Now, can we use these equations in order to find a function $\phi$ and thereafter a function $G$ so that we get a constant of motion?
### Combining the determining equations
Now, we want to combine the third determining equation and the fourth determining equation in order to eliminate the dependence on the arbitrary function $G{(z,u)}$. Denote the first determining equation by $e_{1}$ and the second by $e_{2}$. Then, $e_{1}$ contains the term
$$\exp{(-c\,z)}\dfrac{\partial G}{\partial u}$$
and $e_{2}$ contains the term
$$\exp{(-c\,z)}\dfrac{\partial G}{\partial z}.$$
This implies that $\partial e_{1}/\partial z$ will contain the term
$$\dfrac{\partial}{\partial z}\left(\exp{(-c\,z)}\dfrac{\partial G}{\partial u}\right)=-c\exp{(-c\,z)}\dfrac{\partial G}{\partial u}+\exp{(-c\,z)}\dfrac{\partial^{2} G}{\partial z\partial u}.$$
Similarly, in the case of $e_{2}$, we have that
$$\dfrac{\partial}{\partial u}\left(\exp{(-c\,z)}\dfrac{\partial G}{\partial z}\right)=\exp{(-c\,z)}\dfrac{\partial^{2} G}{\partial u\partial z}.$$
By equating the second order derivatives, i.e. $\partial^{2}G/(\partial z\partial u)=\partial^{2}G/(\partial u\partial z)$, it follows that 
$$\dfrac{\partial}{\partial z}\left(\exp{(-c\,z)}\dfrac{\partial G}{\partial u}\right)+c\exp{(-c\,z)}\dfrac{\partial G}{\partial u}=\dfrac{\partial}{\partial u}\left(\exp{(-c\,z)}\dfrac{\partial G}{\partial z}\right).$$
By linking these three terms to the determining equations $e_{1}$ and $e_{2}$ the following equation must hold
\begin{equation}
\dfrac{\partial e_{1}}{\partial z}+c e_{1}=\dfrac{\partial e_{2}}{\partial u}\,,
\label{eq:combined_equation}
\end{equation}
which is the combined equation that we now will construct. 

In [12]:
# Check if this holds for all arbitrary functions phi and G
det_eq_12 = simplify(Derivative(det_eq[2][1],z).doit()+c*det_eq[2][1]-Derivative(det_eq[3][1],u).doit())
if det_eq_12==0:
    print("We have a Lagrangian for any choices of phi and G!")
else:     
    print("Sorry, we do not have a Lagrangian for any choices of phi and G!")


We have a Lagrangian for any choices of phi and G!


Wow, this is magical! For any, choices of the function $\phi{(u)}$ we have a corresponding $G{(z,u)}$ and thereby a constant of motion! 

Let's derive a few conservation laws!

# Conservation laws for various Lagrangian
Let's look at the choices $\phi{(u)}=0$, $\phi{(u)}=u$ and $\phi{(u)}=u^{2}$ for a start!
### Case 1 out of 3: $\phi{(u)}=0$


In [19]:
# Combined equation
PDE_general = det_eq[2][1]-det_eq[3][1]
# Define our specific PDE
PDE_specific = expand(PDE_general.subs(phi,0).doit())
# Define the PDE to solve
G_pde_1 = Eq(PDE_specific,0)
# Solve the PDE at hand gor the unknown G
G_1 = pdsolve(G_pde_1)
G_1_str = latex(simplify(G_1),mode='equation').replace("\\begin{equation}","\\begin{equation}\n").replace("\\end{equation}","\\,.\n\\end{equation}\n")
print("First value of $G{(z,u)}$ when $\phi{(u)}=%s$"%(0))
print(G_1_str)

First value of $G{(z,u)}$ when $\phi{(u)}=0$
\begin{equation}
G{\left(z,u \right)} = - \frac{\sqrt{6} u^{3} e^{\sqrt{6} z}}{30} + \frac{\sqrt{6} u^{2} e^{\sqrt{6} z}}{30} + F{\left(u + z \right)}\,.
\end{equation}



First value of $G{(z,u)}$ when $\phi{(u)}=0$
\begin{equation}
G{\left(z,u \right)} = - \frac{\sqrt{6} u^{3} e^{\sqrt{6} z}}{30} + \frac{\sqrt{6} u^{2} e^{\sqrt{6} z}}{30} + F{\left(u + z \right)}\,.
\end{equation}
### Case 2 out of 3: $\phi{(u)}=u$


In [20]:
# Define our specific PDE
PDE_specific = expand(PDE_general.subs(phi,u).doit())
# Define the PDE to solve
G_pde_2 = Eq(PDE_specific,0)
# Solve the PDE at hand gor the unknown G
G_2 = pdsolve(G_pde_2)
G_2_str = latex(simplify(G_2),mode='equation').replace("\\begin{equation}","\\begin{equation}\n").replace("\\end{equation}","\\,.\n\\end{equation}\n")
print("Second value of $G{(z,u)}$ when $\phi{(u)}=%s$"%(latex(u)))
print(G_2_str)

Second value of $G{(z,u)}$ when $\phi{(u)}=u$
\begin{equation}
G{\left(z,u \right)} = - \frac{\sqrt{6} u^{3} e^{\sqrt{6} z}}{30} + \frac{\sqrt{6} u^{2} e^{\sqrt{6} z}}{30} - \frac{3 \sqrt{6} u e^{\sqrt{6} z}}{10} + F{\left(u + z \right)}\,.
\end{equation}



Second value of $G{(z,u)}$ when $\phi{(u)}=u$
\begin{equation}
G{\left(z,u \right)} = - \frac{\sqrt{6} u^{3} e^{\sqrt{6} z}}{30} + \frac{\sqrt{6} u^{2} e^{\sqrt{6} z}}{30} - \frac{3 \sqrt{6} u e^{\sqrt{6} z}}{10} + F{\left(u + z \right)}\,.
\end{equation}

### Case 3 out of 3: $\phi{(u)}=u^{2}$






In [22]:
# Define our specific PDE
PDE_specific = expand(PDE_general.subs(phi,u**2).doit())
# Define the PDE to solve
G_pde_3 = Eq(PDE_specific,0)
# Solve the PDE at hand gor the unknown G
G_3 = pdsolve(G_pde_3)
G_3_str = latex(simplify(G_3),mode='equation').replace("\\begin{equation}","\\begin{equation}\n").replace("\\end{equation}","\\,.\n\\end{equation}\n")
print("Third value of $G{(z,u)}$ when $\phi{(u)}=%s$"%(latex(u**2)))
print(G_3_str)

Third value of $G{(z,u)}$ when $\phi{(u)}=u^{2}$
\begin{equation}
G{\left(z,u \right)} = - \frac{\sqrt{6} u^{3} e^{\sqrt{6} z}}{30} - \frac{\sqrt{6} u^{2} e^{\sqrt{6} z}}{15} + F{\left(u + z \right)}\,.
\end{equation}



Third value of $G{(z,u)}$ when $\phi{(u)}=u^{2}$
\begin{equation}
G{\left(z,u \right)} = - \frac{\sqrt{6} u^{3} e^{\sqrt{6} z}}{30} - \frac{\sqrt{6} u^{2} e^{\sqrt{6} z}}{15} + F{\left(u + z \right)}\,.
\end{equation}